In [1]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.0f}'.format

In [2]:
qis_clsf = pd.read_excel('./Downloads/AI060_QIS분류.xlsx')
qis_clsf.head()

,QIS분류,업무보고서
0,NaN,1. 책임보험
1,개인용(인담보),가. 개인용
2,업무용(인담보),나. 업무용
3,영업용(인담보),다. 영업용
4,기타,라. 이륜자동차


In [66]:
def 합산비율(ai060):
    ai060.drop(0, axis=0, inplace=True) # 빈 행 삭제
    ai060 = ai060[:72] # 불량물건 삭제
    ai060.reset_index(drop=True, inplace=True) # 인덱스 리셋
    ai060['QIS분류'] = qis_clsf['QIS분류'] # QIS분류 넣기
    ai060 = ai060[['QIS분류','Unnamed: 0','원수보험료','해약환급금','수재보험료','해약환급금.1','출재보험료','해약환급금환입','Unnamed: 20','발생손해액(c)','금액(d)']] # 필요 컬럼 추출
    ai060['원수보험료'] = ai060['원수보험료'] - ai060['해약환급금']   # 순원수보험료
    ai060['수재보험료'] = ai060['수재보험료'] - ai060['해약환급금.1']   # 순수재보험료
    ai060['출재보험료'] = ai060['출재보험료'] - ai060['해약환급금환입']   # 순출재보험료
    ai060.drop(['해약환급금','해약환급금.1','해약환급금환입'], axis=1, inplace=True) # 해약환급금 컬럼 삭제
    ai060.columns = ['QIS분류','업무보고서분류','원수보험료','수재보험료','출재보험료','경과보험료','발생손해액','순사업비']  # 컬럼명 변경
    ai060 = ai060[~ai060['QIS분류'].isna()] # 불필요한 컬럼 제거
    summary = ai060.groupby('QIS분류').sum()
    row_ord = ['개인용(인담보)','개인용(물담보)','업무용(인담보)','업무용(물담보)','영업용(인담보)','영업용(물담보)','기타']
    summary = summary.loc[row_ord]
    summary.eval('합산비율 = (발생손해액+순사업비)/경과보험료', inplace=True) # 합산비율 계산
    return summary

In [80]:
ai060_1512 = pd.read_excel('./Downloads/AI060_201512.xlsx', sheet_name='AI060', skiprows=11) # 엑셀 읽어오기
ai060_1612 = pd.read_excel('./Downloads/AI060_201612.xlsx', sheet_name='AI060', skiprows=11) # 엑셀 읽어오기
ai060_1712 = pd.read_excel('./Downloads/AI060_201712.xlsx', sheet_name='AI060', skiprows=11) # 엑셀 읽어오기

summary_1512 = 합산비율(ai060_1512) # Output (1)
summary_1612 = 합산비율(ai060_1612) # Output (2)
summary_1712 = 합산비율(ai060_1712) # Output (3)

평균합산비율 = (summary_1512 + summary_1612 + summary_1712)[['합산비율']]/3
평균합산비율.to_excel('output_auto.xlsx')
#평균합산비율.style.format('{:.1%}')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
